<a href="https://colab.research.google.com/github/nts-e/NLP_Parler_Hate_Detection/blob/main/Parler_final_Infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test run the updated BERT model

## Instructions:
1. Use the Presets cell to configure data and model.<br>
2. Run all cells.<br>
3. Collect the result csv from DATA_FOLDER_PATH.<br>

# Imports

In [ ]:
import numpy as np
import pandas as pd
import transformers
import tensorflow as tf
import pickle
from transformers import BartForSequenceClassification, BartTokenizer
import json
from datetime import datetime
from tensorflow.keras import models, layers, metrics, losses, optimizers

# Code

## Presets

In [ ]:
DATA_FOLDER_PATH              = "../Data/Samples/"
MODEL_FOLDER_PATH              = "./"
hate_classifier_threshold = 0.5
bert_model = 'BERT_3'
posts_file = 'parler_data000000000010.notsampled'

## Data

In [ ]:
def load_data(big_dataset_file_name):
  df = pd.read_json(f'{DATA_FOLDER_PATH}{big_dataset_file_name}.ndjson')
  return df

def load_csv(filename):
  df = pd.read_csv(f'{DATA_FOLDER_PATH}{filename}.csv')
  return df


## Classifier

In [ ]:
classifier_tokenizer = None
classifier_model = None

def load_classifier_model():
  # load tokenizer
  global classifier_tokenizer
  base_model_name = 'distilbert-base-uncased'
  classifier_tokenizer = transformers.DistilBertTokenizer.from_pretrained(base_model_name) 

  # load bert model
  config = transformers.DistilBertConfig(dropout=0.2, attention_dropout=0.2) # Nitzan - config here is irrelevant
  dbert_tf = transformers.TFDistilBertModel.from_pretrained(base_model_name, config=config, trainable=False)

  return models.load_model(f'{MODEL_FOLDER_PATH}{bert_model}.h5', custom_objects={'TFDistilBertModel': dbert_tf})

def classify(posts):
  max_length = 190
  X_tokenized = classifier_tokenizer(posts.to_list(), padding='max_length', max_length = max_length, truncation=True, return_attention_mask=True)
  y_pred_proba = classifier_model.predict({'input_ids': np.array(X_tokenized['input_ids']), 'input_attention': np.array(X_tokenized['attention_mask'])})
  # y_pred = np.array([y_pred_proba > hate_classifier_threshold], dtype=int).flatten()
  return y_pred_proba.flatten()


# Tests / Demo

## Init models

In [ ]:
classifier_model = load_classifier_model()

## Load data

In [ ]:
posts = load_csv(posts_file)

## Test

In [ ]:
posts['bert'] = classify(posts['body'])
posts[posts['bert'] > hate_classifier_threshold]
df = pd.DataFrame(data=posts['body'])
df['bert'] = posts['bert']
df.to_csv(f'{DATA_FOLDER_PATH}{posts_file}.res.csv')